In [8]:
!pip3 install pybullet==3.2.5

In [1]:
import pandas as pd
import numpy as np
import pybullet as p
import time
import pybullet_data
import jsonpickle
import os

ASSET_ROOT = os.path.abspath('urdfc')
PLANE_ROOT = os.path.abspath('urdf')

physicsClient = p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.setGravity(0,0,-9.81)

target = (-0.07796166092157364, 0.005451506469398737, -0.06238798052072525)
dist = 1.0
yaw = 89.6000747680664
pitch = -17.800016403198242
p.resetDebugVisualizerCamera(dist,yaw,pitch,target)

planeId = p.loadURDF(os.path.join(PLANE_ROOT, 'plane/plane.urdf'))

origin = np.zeros(3)
noRotation = p.getQuaternionFromEuler([0,0,0])

def load_obj(otype, pos=origin, rot=noRotation, tp=True, wait=0):
    if not isinstance(pos, np.ndarray):
        pos = np.array(pos)
    if not isinstance(rot, np.ndarray):
        rot = p.getQuaternionFromEuler(np.array(pos))
    
    try:
        ret = p.loadURDF(os.path.join(ASSET_ROOT, f'{otype}.urdf'), pos, rot)
    except Exception as e:
        print(otype)
        raise e
    
    for _ in range(wait):
        p.stepSimulation()
        time.sleep(1./240.)
        
    return ret
        
oo1 = load_obj('cuboid', pos=[0, 0, 0.06], wait=100)
oo2 = load_obj('cube', pos=[0, 0.05, 0.05], wait=100)
# o2 = load_obj('cuboid')

prev_line = None

cp = p.getClosestPoints(oo1, oo2, .05)[0]

v1 = np.array(cp[5])
v2 = np.array(cp[6])

if np.linalg.norm(v1) < np.linalg.norm(v2):
    v1, v2 = v2, v1

mp = v1 - v2
mp[-1] = 2

po = load_obj('point', pos=mp)
cp1 = p.getClosestPoints(oo1, po, 5)[0]
cp2 = p.getClosestPoints(oo2, po, 5)[0]

p.addUserDebugLine(cp1[5], cp2[5])

# while True:
#     try:
#         p.stepSimulation()
#         time.sleep(1./240.)
        
#         cp = p.getClosestPoints(oo1, oo2, .05)
#         for x in cp:
#             fr = x[5]
#             to = x[6]
            
#             print(fr, to)
            
#             if prev_line:
#                 p.removeUserDebugItem(prev_line)
#             prev_line = p.addUserDebugLine(fr, to)
        
#     except Exception as e:
#         p.disconnect()
        
#         raise e
        
# for _ in range(500):
#     try:
#         p.stepSimulation()
#         time.sleep(1./240.)
        
#     except Exception as e:
#         p.disconnect()
        
#         raise e    


# ray_count = 20

# cf, oc1, oc2, _, _, posoc1, posoc2, *_ = p.getClosestPoints(oo1, oo2, 0.6)[0]


# for o1, pos in zip([oc1, oc2], [posoc1, posoc2]):
# #     pos = p.getBasePositionAndOrientation(o1)[0]
#     x = np.linspace(pos[0]-.035, pos[0]+0.035, ray_count)
#     y = np.linspace(pos[1]-.035, pos[1]+0.035, ray_count)

#     xx, yy = [a.flatten() for a in np.meshgrid(x, y)]

#     rays_fr = list(zip(xx, yy, np.full(ray_count**2, 10)))
#     rays_to = list(zip(xx, yy, np.full(ray_count**2, 0)))

#     hits = p.rayTestBatch(rays_fr, rays_to, numThreads=0)

# #     if prev_line:

# #         for l in prev_line:
# #             p.removeUserDebugItem(l)

#     points = []

#     for ho, li, hf, hpos, hnor in hits:
#         if ho != o1:
#             continue

#         points.append(hpos)

#     prev_line = []
#     for p1 in points:
#         prev_line.append(p.addUserDebugLine(p1, np.array(p1)+0.001))
        
# print(p.MAX_RAY_INTERSECTION_BATCH_SIZE)

pybullet build time: Nov 30 2022 16:56:59


0